In [ ]:
# ============================================================================
# THE AUTONOMOUS COLONY - PART 3: MULTI-AGENT RL
# Cooperation, Competition, and Emergent Behaviors
# ============================================================================

"""
# 🤝 The Autonomous Colony - Multi-Agent Coordination

**RL Concepts Covered:**
1. Multi-Agent RL (MARL) fundamentals
2. Communication between agents
3. Centralized training, decentralized execution (CTDE)
4. Reward shaping for cooperation
5. Nash equilibria and social dilemmas
6. Parameter sharing vs independent learners
7. PettingZoo integration

**Prerequisites:**
- Parts 1 & 2 (environment and single-agent RL)
"""

# ============================================================================
# SETUP
# ============================================================================

# !pip install pettingzoo supersuit torch -q

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from collections import deque

print("✓ Multi-agent libraries loaded")

# ============================================================================
# 1. COMMUNICATION MODULE
# ============================================================================

class CommunicationNetwork(nn.Module):
    """
    Learned communication between agents.
    Each agent can send/receive messages to nearby agents.
    
    RL Concept: Communication as part of the action space
    """
    
    def __init__(self, message_dim: int = 16, hidden_dim: int = 64):
        super().__init__()
        self.message_dim = message_dim
        
        # Message encoder (from agent state to message)
        self.encoder = nn.Sequential(
            nn.Linear(5, hidden_dim),  # 5 = agent state features
            nn.ReLU(),
            nn.Linear(hidden_dim, message_dim)
        )
        
        # Message aggregator (combine received messages)
        self.aggregator = nn.Sequential(
            nn.Linear(message_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
    
    def encode_message(self, agent_state):
        """Generate message from agent state"""
        return self.encoder(agent_state)
    
    def aggregate_messages(self, messages):
        """Aggregate messages from multiple agents"""
        if len(messages) == 0:
            return torch.zeros(1, self.message_dim)
        
        # Mean pooling
        combined = torch.stack(messages).mean(dim=0)
        return self.aggregator(combined)

# ============================================================================
# 2. MULTI-AGENT ACTOR-CRITIC
# ============================================================================

class MultiAgentActorCritic(nn.Module):
    """
    Actor-Critic with communication for multi-agent settings.
    
    Architecture:
    - Individual observations + messages -> features
    - Actor: decentralized policy per agent
    - Critic: can use global state (CTDE)
    """
    
    def __init__(
        self,
        grid_shape: tuple,
        state_dim: int,
        action_dim: int,
        n_agents: int,
        message_dim: int = 16,
        use_communication: bool = True
    ):
        super().__init__()
        self.n_agents = n_agents
        self.use_communication = use_communication
        
        # Communication module
        if use_communication:
            self.comm = CommunicationNetwork(message_dim)
            extra_dim = message_dim
        else:
            extra_dim = 0
        
        # Individual feature extractors (decentralized)
        self.conv1 = nn.Conv2d(grid_shape[2], 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        conv_out = grid_shape[0] * grid_shape[1] * 64
        
        self.state_fc = nn.Linear(state_dim, 64)
        self.feature_fc = nn.Linear(conv_out + 64 + extra_dim, 256)
        
        # Actor (decentralized - per agent policy)
        self.actor = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
        
        # Critic (centralized - global value)
        # Takes features from all agents
        self.critic_fc = nn.Linear(256 * n_agents, 256)
        self.critic_out = nn.Linear(256, 1)
    
    def forward_actor(self, grid_obs, state_obs, messages=None):
        """Forward pass for actor (decentralized)"""
        # Extract features
        x = F.relu(self.conv1(grid_obs))
        x = F.relu(self.conv2(x))
        x = x.flatten(1)
        s = F.relu(self.state_fc(state_obs))
        
        # Add communication
        if self.use_communication and messages is not None:
            features = torch.cat([x, s, messages], dim=1)
        else:
            features = torch.cat([x, s], dim=1)
        
        features = F.relu(self.feature_fc(features))
        logits = self.actor(features)
        
        return logits, features
    
    def forward_critic(self, all_agent_features):
        """Forward pass for critic (centralized)"""
        # Concatenate all agent features
        combined = torch.cat(all_agent_features, dim=1)
        x = F.relu(self.critic_fc(combined))
        value = self.critic_out(x)
        return value
    
    def get_actions_and_value(self, observations, training=True):
        """
        Get actions for all agents and global value.
        
        CTDE: Centralized training (uses all observations for value),
              Decentralized execution (each agent acts on local obs)
        """
        batch_size = observations[0]['grid'].shape[0]
        
        # Step 1: Generate messages (if using communication)
        messages_all = []
        if self.use_communication and training:
            for obs in observations:
                state = obs['state']
                message = self.comm.encode_message(state)
                messages_all.append(message)
        
        # Step 2: Process each agent
        actions = []
        log_probs = []
        entropies = []
        features_all = []
        
        for i, obs in enumerate(observations):
            grid = obs['grid'].permute(0, 3, 1, 2)  # NHWC -> NCHW
            state = obs['state']
            
            # Aggregate messages from other agents
            if self.use_communication and training:
                # Simple: aggregate all other agents' messages
                other_messages = [messages_all[j] for j in range(self.n_agents) if j != i]
                if other_messages:
                    aggregated_msg = torch.stack(other_messages).mean(dim=0)
                else:
                    aggregated_msg = torch.zeros(batch_size, self.comm.message_dim).to(grid.device)
            else:
                aggregated_msg = None
            
            # Get policy
            logits, features = self.forward_actor(grid, state, aggregated_msg)
            probs = torch.softmax(logits, dim=-1)
            dist = torch.distributions.Categorical(probs)
            
            action = dist.sample()
            log_prob = dist.log_prob(action)
            entropy = dist.entropy()
            
            actions.append(action)
            log_probs.append(log_prob)
            entropies.append(entropy)
            features_all.append(features)
        
        # Step 3: Centralized value (uses all agent features)
        if training:
            value = self.forward_critic(features_all)
        else:
            value = None
        
        return actions, log_probs, entropies, value

# ============================================================================
# 3. MULTI-AGENT PPO
# ============================================================================

class MultiAgentPPO:
    """
    PPO for multi-agent coordination.
    
    Key features:
    - Shared parameters across agents (faster learning)
    - Communication between agents
    - Cooperative reward shaping
    - Centralized critic, decentralized actors (CTDE)
    """
    
    def __init__(
        self,
        grid_shape: tuple,
        state_dim: int,
        action_dim: int,
        n_agents: int,
        learning_rate: float = 3e-4,
        gamma: float = 0.99,
        gae_lambda: float = 0.95,
        clip_epsilon: float = 0.2,
        entropy_coef: float = 0.01,
        value_coef: float = 0.5,
        cooperation_bonus: float = 2.0,
        use_communication: bool = True
    ):
        self.n_agents = n_agents
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.clip_epsilon = clip_epsilon
        self.entropy_coef = entropy_coef
        self.value_coef = value_coef
        self.cooperation_bonus = cooperation_bonus
        
        # Network
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.network = MultiAgentActorCritic(
            grid_shape, state_dim, action_dim, n_agents,
            use_communication=use_communication
        ).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)
        
        # Rollout buffer
        self.rollout_buffer = []
        self.loss_history = []
        
        print(f"✓ Multi-Agent PPO initialized ({n_agents} agents, comm={use_communication})")
    
    def select_actions(self, observations: List[Dict]):
        """Select actions for all agents"""
        # Convert observations to tensors
        obs_tensors = []
        for obs in observations:
            grid = torch.FloatTensor(obs['grid']).unsqueeze(0).to(self.device)
            state = torch.FloatTensor(obs['state']).unsqueeze(0).to(self.device)
            obs_tensors.append({'grid': grid, 'state': state})
        
        with torch.no_grad():
            actions, log_probs, entropies, value = self.network.get_actions_and_value(
                obs_tensors, training=True
            )
        
        actions_np = [a.item() for a in actions]
        log_probs_np = [lp.item() for lp in log_probs]
        value_np = value.item() if value is not None else 0.0
        
        return actions_np, log_probs_np, value_np
    
    def compute_cooperative_reward(self, individual_rewards: List[float], 
                                   agents_alive: List[bool]) -> List[float]:
        """
        Shape rewards to encourage cooperation.
        
        Strategies:
        - Team bonus: reward when all agents survive
        - Proximity bonus: reward agents near each other
        - Resource sharing: higher reward when resources distributed
        """
        n_alive = sum(agents_alive)
        team_bonus = self.cooperation_bonus * (n_alive / self.n_agents)
        
        shaped_rewards = []
        for i, (reward, alive) in enumerate(zip(individual_rewards, agents_alive)):
            if alive:
                # Add team survival bonus
                shaped_rewards.append(reward + team_bonus)
            else:
                shaped_rewards.append(reward)
        
        return shaped_rewards
    
    def store_transition(self, observations, actions, rewards, log_probs, value, dones):
        """Store multi-agent transition"""
        self.rollout_buffer.append({
            'observations': observations,
            'actions': actions,
            'rewards': rewards,
            'log_probs': log_probs,
            'value': value,
            'dones': dones
        })
    
    def update(self, n_epochs: int = 4):
        """PPO update for multi-agent setting"""
        if len(self.rollout_buffer) < 32:
            return None
        
        # Extract data
        all_obs = [t['observations'] for t in self.rollout_buffer]
        all_actions = [t['actions'] for t in self.rollout_buffer]
        all_rewards = [t['rewards'] for t in self.rollout_buffer]
        all_log_probs = [t['log_probs'] for t in self.rollout_buffer]
        all_values = [t['value'] for t in self.rollout_buffer]
        all_dones = [t['dones'] for t in self.rollout_buffer]
        
        # Compute advantages (using team value)
        advantages = []
        returns = []
        gae = 0
        
        for t in reversed(range(len(all_rewards))):
            # Use mean reward across agents
            mean_reward = np.mean(all_rewards[t])
            mean_done = any(all_dones[t])
            
            if t == len(all_rewards) - 1:
                next_value = 0
            else:
                next_value = all_values[t + 1]
            
            delta = mean_reward + self.gamma * next_value * (1 - mean_done) - all_values[t]
            gae = delta + self.gamma * self.gae_lambda * (1 - mean_done) * gae
            
            advantages.insert(0, gae)
            returns.insert(0, gae + all_values[t])
        
        # Normalize
        advantages = np.array(advantages)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # Convert to tensors (flattened across agents)
        obs_grids = []
        obs_states = []
        actions_flat = []
        old_log_probs_flat = []
        
        for t in range(len(all_obs)):
            for agent_id in range(self.n_agents):
                obs = all_obs[t][agent_id]
                obs_grids.append(torch.FloatTensor(obs['grid']))
                obs_states.append(torch.FloatTensor(obs['state']))
                actions_flat.append(all_actions[t][agent_id])
                old_log_probs_flat.append(all_log_probs[t][agent_id])
        
        # Batch tensors
        grids_batch = torch.stack(obs_grids).to(self.device)
        states_batch = torch.stack(obs_states).to(self.device)
        actions_batch = torch.LongTensor(actions_flat).to(self.device)
        old_log_probs_batch = torch.FloatTensor(old_log_probs_flat).to(self.device)
        advantages_batch = torch.FloatTensor(advantages).repeat_interleave(self.n_agents).to(self.device)
        returns_batch = torch.FloatTensor(returns).repeat_interleave(self.n_agents).to(self.device)
        
        # PPO epochs
        total_loss = 0
        for epoch in range(n_epochs):
            # Forward pass for all agents
            # (Simplified: process in batch rather than grouping by timestep)
            logits, features = self.network.forward_actor(
                grids_batch.permute(0, 3, 1, 2),
                states_batch
            )
            
            probs = torch.softmax(logits, dim=-1)
            dist = torch.distributions.Categorical(probs)
            new_log_probs = dist.log_prob(actions_batch)
            entropy = dist.entropy()
            
            # Value function (simplified: use per-agent value)
            values_pred = self.network.forward_critic([features] * self.n_agents).squeeze()
            
            # PPO loss
            ratio = torch.exp(new_log_probs - old_log_probs_batch)
            surr1 = ratio * advantages_batch
            surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * advantages_batch
            policy_loss = -torch.min(surr1, surr2).mean()
            
            value_loss = F.mse_loss(values_pred, returns_batch)
            entropy_loss = -entropy.mean()
            
            loss = policy_loss + self.value_coef * value_loss + self.entropy_coef * entropy_loss
            
            # Optimize
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
            self.optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / n_epochs
        self.loss_history.append(avg_loss)
        
        # Clear buffer
        self.rollout_buffer = []
        
        return avg_loss

# ============================================================================
# 4. TRAINING LOOP FOR MULTI-AGENT
# ============================================================================

def train_multiagent(env, agent, n_episodes: int = 200):
    """Train multi-agent system"""
    episode_rewards = []
    survival_rates = []
    cooperation_scores = []
    
    print(f"\n🚀 Training Multi-Agent System for {n_episodes} episodes...")
    
    for episode in range(n_episodes):
        observations, info = env.reset()
        done_flags = [False] * env.config.n_agents
        episode_reward = [0.0] * env.config.n_agents
        step_count = 0
        
        while not all(done_flags) and step_count < env.config.max_steps:
            # Get actions for all agents
            actions, log_probs, value = agent.select_actions(observations)
            
            # Step environment
            next_observations, rewards, dones, truncated, info = env.step(actions)
            
            # Compute cooperative rewards
            agents_alive = [not (d or t) for d, t in zip(dones, truncated)]
            shaped_rewards = agent.compute_cooperative_reward(rewards, agents_alive)
            
            # Store transition
            agent.store_transition(
                observations, actions, shaped_rewards, log_probs, value,
                [d or t for d, t in zip(dones, truncated)]
            )
            
            # Update episode rewards
            for i in range(env.config.n_agents):
                episode_reward[i] += shaped_rewards[i]
                done_flags[i] = dones[i] or truncated[i]
            
            observations = next_observations
            step_count += 1
        
        # Update agent
        if episode % 5 == 0:  # Update every 5 episodes
            loss = agent.update(n_epochs=4)
            if loss:
                print(f"  Update loss: {loss:.4f}")
        
        # Metrics
        mean_reward = np.mean(episode_reward)
        survival_rate = sum(not d for d in done_flags) / len(done_flags)
        cooperation_score = mean_reward * survival_rate  # Combined metric
        
        episode_rewards.append(mean_reward)
        survival_rates.append(survival_rate)
        cooperation_scores.append(cooperation_score)
        
        # Logging
        if (episode + 1) % 20 == 0:
            avg_reward = np.mean(episode_rewards[-20:])
            avg_survival = np.mean(survival_rates[-20:])
            avg_coop = np.mean(cooperation_scores[-20:])
            print(f"\nEpisode {episode+1}/{n_episodes}")
            print(f"  Avg Reward: {avg_reward:.2f}")
            print(f"  Avg Survival: {avg_survival:.2%}")
            print(f"  Cooperation Score: {avg_coop:.2f}")
    
    return {
        'rewards': episode_rewards,
        'survival_rates': survival_rates,
        'cooperation_scores': cooperation_scores
    }

# ============================================================================
# 5. VISUALIZATION
# ============================================================================

def plot_multiagent_results(results: Dict):
    """Plot multi-agent training metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Mean rewards
    rewards = results['rewards']
    axes[0, 0].plot(rewards, alpha=0.3, label='Raw')
    if len(rewards) > 20:
        smoothed = np.convolve(rewards, np.ones(20)/20, mode='valid')
        axes[0, 0].plot(range(19, len(rewards)), smoothed, linewidth=2, label='Smoothed')
    axes[0, 0].set_xlabel('Episode')
    axes[0, 0].set_ylabel('Mean Reward')
    axes[0, 0].set_title('Multi-Agent Mean Rewards')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Survival rates
    survival = results['survival_rates']
    axes[0, 1].plot(survival, alpha=0.3, label='Raw')
    if len(survival) > 20:
        smoothed = np.convolve(survival, np.ones(20)/20, mode='valid')
        axes[0, 1].plot(range(19, len(survival)), smoothed, linewidth=2, label='Smoothed')
    axes[0, 1].set_xlabel('Episode')
    axes[0, 1].set_ylabel('Survival Rate')
    axes[0, 1].set_title('Agent Survival Rate')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([0, 1.1])
    
    # Cooperation scores
    coop = results['cooperation_scores']
    axes[1, 0].plot(coop, alpha=0.3, label='Raw')
    if len(coop) > 20:
        smoothed = np.convolve(coop, np.ones(20)/20, mode='valid')
        axes[1, 0].plot(range(19, len(coop)), smoothed, linewidth=2, label='Smoothed')
    axes[1, 0].set_xlabel('Episode')
    axes[1, 0].set_ylabel('Cooperation Score')
    axes[1, 0].set_title('Cooperation Score (Reward × Survival)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Distribution comparison (final 50 episodes)
    axes[1, 1].hist(rewards[-50:], alpha=0.5, bins=20, label='Rewards')
    axes[1, 1].hist(coop[-50:], alpha=0.5, bins=20, label='Cooperation')
    axes[1, 1].set_xlabel('Score')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Score Distribution (Last 50 Episodes)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle('Multi-Agent Training Results', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# ============================================================================
# DEMO & EXPERIMENTS
# ============================================================================

print("\n" + "="*80)
print("MULTI-AGENT RL MODULE READY!")
print("="*80)
print("\n📊 Key Concepts Implemented:")
print("  ✓ Multi-agent actor-critic architecture")
print("  ✓ Agent communication network")
print("  ✓ Centralized training, decentralized execution (CTDE)")
print("  ✓ Cooperative reward shaping")
print("  ✓ Parameter sharing across agents")
print("\n🧪 Experiments to try:")
print("  1. With vs without communication")
print("  2. Different cooperation bonus values")
print("  3. Competitive vs cooperative reward structures")
print("  4. Emergent behaviors and specialization")
print("="*80)

# Example usage (uncomment to run):
"""
# Create environment
from part1_environment import ColonyEnvironment, EnvironmentConfig

config = EnvironmentConfig(
    grid_size=20,
    n_agents=4,
    max_steps=300
)
env = ColonyEnvironment(config)

# Create multi-agent PPO
ma_agent = MultiAgentPPO(
    grid_shape=(7, 7, 5),  # observation_radius=3
    state_dim=5,
    action_dim=9,
    n_agents=4,
    use_communication=True,
    cooperation_bonus=3.0
)

# Train
results = train_multiagent(env, ma_agent, n_episodes=200)
plot_multiagent_results(results)

# Visualize trained agents
env.render()
"""